In [2]:
!conda install -c conda-forge geopy --yes 

!pip install geopy
!pip install geocoder
!pip install folium

from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

from geopy.geocoders import Nominatim
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score

%matplotlib notebook

import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
print('Folium installed')
print('Libraries imported.')
print('All libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.
All libraries imported.


In [17]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Cape_Town_suburbs')[-1]
df.columns = ['district', 'suburb', 'c']
df = df.drop(['c'],axis =1)
#df = df.iloc[1: , :]
df.head(10)

,district,suburb
0,Seat: Cape Town,Seat: Cape Town
1,Atlantic Seaboard,Bakoven Bantry Bay Camps Bay Clifton Fresnaye ...
2,Blaauwberg,Atlantis Bloubergstrand Brooklyn Century City ...
3,Cape Flats,Athlone Bishop Lavis Crawford Crossroads Gugul...
4,City Centre,Bo-Kaap De Waterkant Devil's Peak Estate Distr...
5,Helderberg,Eerste River Faure Gordon's Bay Macassar Sir L...
6,Northern Suburbs,Belhar Bellville Boston Eversdal Kenridge Stel...
7,Southern Suburbs,Bergvliet Bishopscourt Claremont Constantia Di...
8,South Peninsula,Capri Village Clovelly Da Gama Park Fish Hoek ...


In [18]:
df['Latitude1'] = None
df['Longitude1'] = None

for i, neigh in enumerate(df['district']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Cape Town, South Africa'.format(neigh))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    df.loc[i, 'Latitude1'] = latitude
    df.loc[i, 'Longitude1'] = longitude

df.head(10)

,district,suburb,Latitude1,Longitude1
0,Seat: Cape Town,Seat: Cape Town,-33.9191,18.422
1,Atlantic Seaboard,Bakoven Bantry Bay Camps Bay Clifton Fresnaye ...,-33.9123,18.4164
2,Blaauwberg,Atlantis Bloubergstrand Brooklyn Century City ...,-33.7673,18.4622
3,Cape Flats,Athlone Bishop Lavis Crawford Crossroads Gugul...,-33.9362,18.6161
4,City Centre,Bo-Kaap De Waterkant Devil's Peak Estate Distr...,-33.917,18.4244
5,Helderberg,Eerste River Faure Gordon's Bay Macassar Sir L...,-34.1021,18.8727
6,Northern Suburbs,Belhar Bellville Boston Eversdal Kenridge Stel...,-33.9858,18.6825
7,Southern Suburbs,Bergvliet Bishopscourt Claremont Constantia Di...,-33.9775,18.483
8,South Peninsula,Capri Village Clovelly Da Gama Park Fish Hoek ...,-34.0301,18.451


In [68]:
# create map of Toronto using latitude and longitude values
dist_ct = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude1'], df['Longitude1'], df['district']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(dist_ct)  
    
dist_ct

In [19]:
address = 'Cape Town, Western Cape, South Africa'

geolocator = Nominatim(user_agent="course_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Cape Town, Western Cape, South Africa are {}, {}.'.format(latitude, longitude))


The geographical coordinates of Cape Town, Western Cape, South Africa are -33.928992, 18.417396.


In [20]:
vac_ct = pd.read_csv("/Users/DrumTribe/Desktop/Vac_CapeTown.csv")
vac_ct

,Province,District,Facility,Public/Private
0,Western Cape,Cape Town MM,Adriaanse Clinic,Public
1,Western Cape,Cape Town MM,Bishop Lavis CDC,Public
2,Western Cape,Cape Town MM,Bothasig CDC,Public
3,Western Cape,Cape Town MM,Brackenfell Clinic,Public
4,Western Cape,Cape Town MM,Brooklyn Chest Hospital,Public
...,...,...,...,...
69,Western Cape,Cape Town MM,Stikland Hospital,Public
70,Western Cape,Cape Town MM,Symphony Way CDC,Public
71,Western Cape,Cape Town MM,Tafelsig CDC,Public
72,Western Cape,Cape Town MM,Vanguard CHC,Public


In [21]:
vac_ct['Latitude1'] = None
vac_ct['Longitude1'] = None

for i, neigh in enumerate(vac_ct['Facility']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Cape Town, South Africa'.format(neigh))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    vac_ct.loc[i, 'Latitude1'] = latitude
    vac_ct.loc[i, 'Longitude1'] = longitude

vac_ct.head(10)

,Province,District,Facility,Public/Private,Latitude1,Longitude1
0,Western Cape,Cape Town MM,Adriaanse Clinic,Public,-33.9374,18.5842
1,Western Cape,Cape Town MM,Bishop Lavis CDC,Public,-33.9479,18.5808
2,Western Cape,Cape Town MM,Bothasig CDC,Public,-33.8578,18.5392
3,Western Cape,Cape Town MM,Brackenfell Clinic,Public,-33.8769,18.6883
4,Western Cape,Cape Town MM,Brooklyn Chest Hospital,Public,-33.9007,18.489
5,Western Cape,Cape Town MM,Clicks Pharmacy Blue Route,Private,-34.0659,18.4567
6,Western Cape,Cape Town MM,Clicks Pharmacy Constantia,Private,-34.0219,18.4432
7,Western Cape,Cape Town MM,Clicks Pharmacy Edgemead,Private,-33.8718,18.5473
8,Western Cape,Cape Town MM,Clicks Pharmacy Tygervalley,Private,-33.8737,18.6342
9,Western Cape,Cape Town MM,Clicks Pharmacy Westcoast Village,Private,-33.8047,18.4844


In [22]:
# create map of Toronto using latitude and longitude values
map_ct = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(vac_ct['Latitude1'], vac_ct['Longitude1'], vac_ct['Facility']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ct)  
    
map_ct

In [23]:
# create map of Toronto using latitude and longitude values
# add markers to map
for lat, lng, neighborhood in zip(df['Latitude1'], df['Longitude1'], df['district']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_ct)  
    
map_ct

In [24]:
CLIENT_ID = 'H1UCPKFE1SJC1OXGY5PQNG133KVSKYQCDRXMLFYE04AM00PO' # your Foursquare ID
CLIENT_SECRET = 'FOEUOF2Q5UAKRMOPG0IOGFHX0K3MTZ0C5IFQLAOJPR3PDG1X' # your Foursquare Secret
ACCESS_TOKEN = '4GVDWKVZZFFTCM5FEQS4HG5TNH04Q5MADW21TDBX14BTH302' # Access Token
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: H1UCPKFE1SJC1OXGY5PQNG133KVSKYQCDRXMLFYE04AM00PO
CLIENT_SECRET:FOEUOF2Q5UAKRMOPG0IOGFHX0K3MTZ0C5IFQLAOJPR3PDG1X
ACCESS_TOKEN:4GVDWKVZZFFTCM5FEQS4HG5TNH04Q5MADW21TDBX14BTH302


In [25]:
search_query = 'Pharmacy'
radius = 5000
print(search_query + ' .... OK!')

Pharmacy .... OK!


###### Atlantic Seaboard

In [26]:
address = 'Atlantic Seaboard, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-33.9098767 18.40850280558061


In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=H1UCPKFE1SJC1OXGY5PQNG133KVSKYQCDRXMLFYE04AM00PO&client_secret=FOEUOF2Q5UAKRMOPG0IOGFHX0K3MTZ0C5IFQLAOJPR3PDG1X&ll=-33.9098767,18.40850280558061&oauth_token=4GVDWKVZZFFTCM5FEQS4HG5TNH04Q5MADW21TDBX14BTH302&v=20180604&query=Pharmacy&radius=5000&limit=100'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d9dd4e11af0c6267e1bcf9'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4bc3239ab492d13ac84ba860',
    'name': 'Clicks Pharmacy',
    'location': {'address': 'Burg St.',
     'crossStreet': 'Church St.',
     'lat': -33.92363424450691,
     'lng': 18.419296964562147,
     'labeledLatLngs': [{'label': 'display',
       'lat': -33.92363424450691,
       'lng': 18.419296964562147}],
     'distance': 1827,
     'postalCode': '8000',
     'cc': 'ZA',
     'city': 'iKapa',
     'state': 'Western Cape',
     'country': 'iNingizimu Afrika',
     'formattedAddress': ['Burg St. (Church St.)',
      'iKapa',
      '8000',
      'iNingizimu Afrika']},
    'categories': [{'id': '4bf58dd8d48988d10f951735',
      'name': 'Pharmacy',
      'pluralName': 'Pharmacies',
      'shortName': 'Pharmacy',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_',
       'suffix': '.p

In [29]:
venues = results['response']['venues']

pharmacy = json_normalize(venues)
pharmacy.head()

<ipython-input-29-4f9ec9a50e77>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  pharmacy = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bc3239ab492d13ac84ba860,Clicks Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,Burg St.,Church St.,-33.923634,18.419297,"[{'label': 'display', 'lat': -33.9236342445069...",1827,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Burg St. (Church St.), iKapa, 8000, iNingizim...",NaN
1,4c10feb181e976b001fe0feb,Hypermed Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,"91 Main Road, Doric Crt, Green Point",York St,-33.908215,18.409575,"[{'label': 'display', 'lat': -33.9082145690918...",209,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[91 Main Road, Doric Crt, Green Point (York St...",NaN
2,5a5740f0364d9712b2a78a9b,Dis-Chem Pharmacy The Point Centre,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,76 Regent Road,NaN,-33.920890,18.383193,"[{'label': 'display', 'lat': -33.9208901, 'lng...",2640,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[76 Regent Road, iKapa, 8005, iNingizimu Afrika]",NaN
3,58a3bdb8c30f7936edefc851,Cape Quarter Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,"27 Somerset Road, Cape Quarter, Green Point",NaN,-33.912914,18.416883,"[{'label': 'display', 'lat': -33.9129142761230...",844,8005,ZA,Green Point,Western Cape,iNingizimu Afrika,"[27 Somerset Road, Cape Quarter, Green Point, ...",NaN
4,4e16ec1ae4cd70c587bc6ae4,Waterfront Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,NaN,NaN,-33.903955,18.420108,"[{'label': 'display', 'lat': -33.9039547868217...",1258,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN


In [30]:
pharm_clean=pharmacy.rename(columns={'location.lat': 'lat','location.lng':'long'})
pharm_clean.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,lat,long,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bc3239ab492d13ac84ba860,Clicks Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,Burg St.,Church St.,-33.923634,18.419297,"[{'label': 'display', 'lat': -33.9236342445069...",1827,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Burg St. (Church St.), iKapa, 8000, iNingizim...",NaN
1,4c10feb181e976b001fe0feb,Hypermed Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,"91 Main Road, Doric Crt, Green Point",York St,-33.908215,18.409575,"[{'label': 'display', 'lat': -33.9082145690918...",209,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[91 Main Road, Doric Crt, Green Point (York St...",NaN
2,5a5740f0364d9712b2a78a9b,Dis-Chem Pharmacy The Point Centre,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,76 Regent Road,NaN,-33.920890,18.383193,"[{'label': 'display', 'lat': -33.9208901, 'lng...",2640,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[76 Regent Road, iKapa, 8005, iNingizimu Afrika]",NaN
3,58a3bdb8c30f7936edefc851,Cape Quarter Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,"27 Somerset Road, Cape Quarter, Green Point",NaN,-33.912914,18.416883,"[{'label': 'display', 'lat': -33.9129142761230...",844,8005,ZA,Green Point,Western Cape,iNingizimu Afrika,"[27 Somerset Road, Cape Quarter, Green Point, ...",NaN
4,4e16ec1ae4cd70c587bc6ae4,Waterfront Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624890702,False,NaN,NaN,-33.903955,18.420108,"[{'label': 'display', 'lat': -33.9039547868217...",1258,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN


In [31]:
Pharmacy_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(pharm_clean.lat, pharm_clean.long, pharm_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)

# display map
Pharmacy_map

In [88]:
#2 

address = 'Blaauwberg, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

Bl = json_normalize(venues)
Bl.head()

Bl_clean=Bl.rename(columns={'location.lat': 'lat','location.lng':'long'})
Bl_clean.head(5)

for lat, lng, label in zip(Bl_clean.lat, Bl_clean.long, Bl_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)

-33.8335912 18.522135


<ipython-input-88-4c90044cb65f>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Bl = json_normalize(venues)


In [98]:
address = 'Blaauwberg, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-33.8335912 18.522135


In [99]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=H1UCPKFE1SJC1OXGY5PQNG133KVSKYQCDRXMLFYE04AM00PO&client_secret=FOEUOF2Q5UAKRMOPG0IOGFHX0K3MTZ0C5IFQLAOJPR3PDG1X&ll=-33.8335912,18.522135&oauth_token=4GVDWKVZZFFTCM5FEQS4HG5TNH04Q5MADW21TDBX14BTH302&v=20180604&query=Pharmacy&radius=5000&limit=100'

In [100]:
venues = results['response']['venues']

In [101]:
Bl = json_normalize(venues)
Bl.head()

<ipython-input-101-4dcc5406341f>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Bl = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bc3239ab492d13ac84ba860,Clicks Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,Burg St.,Church St.,-33.923634,18.419297,"[{'label': 'display', 'lat': -33.9236342445069...",1827,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Burg St. (Church St.), iKapa, 8000, iNingizim...",NaN
1,4c10feb181e976b001fe0feb,Hypermed Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"91 Main Road, Doric Crt, Green Point",York St,-33.908215,18.409575,"[{'label': 'display', 'lat': -33.9082145690918...",209,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[91 Main Road, Doric Crt, Green Point (York St...",NaN
2,5a5740f0364d9712b2a78a9b,Dis-Chem Pharmacy The Point Centre,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,76 Regent Road,NaN,-33.920890,18.383193,"[{'label': 'display', 'lat': -33.9208901, 'lng...",2640,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[76 Regent Road, iKapa, 8005, iNingizimu Afrika]",NaN
3,58a3bdb8c30f7936edefc851,Cape Quarter Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"27 Somerset Road, Cape Quarter, Green Point",NaN,-33.912914,18.416883,"[{'label': 'display', 'lat': -33.9129142761230...",844,8005,ZA,Green Point,Western Cape,iNingizimu Afrika,"[27 Somerset Road, Cape Quarter, Green Point, ...",NaN
4,4e16ec1ae4cd70c587bc6ae4,Waterfront Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,NaN,NaN,-33.903955,18.420108,"[{'label': 'display', 'lat': -33.9039547868217...",1258,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN


In [102]:
Bl_clean=Bl.rename(columns={'location.lat': 'lat','location.lng':'long'})
Bl_clean.head(50)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,lat,long,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bc3239ab492d13ac84ba860,Clicks Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,Burg St.,Church St.,-33.923634,18.419297,"[{'label': 'display', 'lat': -33.9236342445069...",1827,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Burg St. (Church St.), iKapa, 8000, iNingizim...",NaN
1,4c10feb181e976b001fe0feb,Hypermed Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"91 Main Road, Doric Crt, Green Point",York St,-33.908215,18.409575,"[{'label': 'display', 'lat': -33.9082145690918...",209,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[91 Main Road, Doric Crt, Green Point (York St...",NaN
2,5a5740f0364d9712b2a78a9b,Dis-Chem Pharmacy The Point Centre,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,76 Regent Road,NaN,-33.920890,18.383193,"[{'label': 'display', 'lat': -33.9208901, 'lng...",2640,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[76 Regent Road, iKapa, 8005, iNingizimu Afrika]",NaN
3,58a3bdb8c30f7936edefc851,Cape Quarter Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"27 Somerset Road, Cape Quarter, Green Point",NaN,-33.912914,18.416883,"[{'label': 'display', 'lat': -33.9129142761230...",844,8005,ZA,Green Point,Western Cape,iNingizimu Afrika,"[27 Somerset Road, Cape Quarter, Green Point, ...",NaN
4,4e16ec1ae4cd70c587bc6ae4,Waterfront Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,NaN,NaN,-33.903955,18.420108,"[{'label': 'display', 'lat': -33.9039547868217...",1258,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN
5,4d0e0f353bbaa14335a240da,Table Bay Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,Breakwater Ln,at Granger Rd,-33.903263,18.420961,"[{'label': 'display', 'lat': -33.9032625009090...",1366,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Breakwater Ln (at Granger Rd), iKapa, iNingiz...",NaN
6,5324429c498eff1bc939fef4,Olsen's Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,129 Long St,NaN,-33.923032,18.415895,"[{'label': 'display', 'lat': -33.9230321540706...",1615,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[129 Long St, iKapa, 8000, iNingizimu Afrika]",NaN
7,4e32d27ba8097f47f4a2142c,Barr's Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,Waterkant St,NaN,-33.920251,18.422334,"[{'label': 'display', 'lat': -33.920251, 'lng'...",1722,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Waterkant St, iKapa, iNingizimu Afrika]",NaN
8,4bf2b64455c7c9b61d656204,Lite Kem Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,NaN,NaN,-33.924460,18.422907,"[{'label': 'display', 'lat': -33.9244596071952...",2099,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN
9,4c6a6a9d1a6620a180f7658c,Tothill's Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,14 Lower Burg St,Riebeeck Street,-33.920013,18.423197,"[{'label': 'display', 'lat': -33.9200134984192...",1765,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[14 Lower Burg St (Riebeeck Street), iKapa, iN...",NaN


In [69]:
#3

address = 'Cape Flats, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = -33.917
longitude = 18.4244
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

CF = json_normalize(venues)
CF.head()

CF_clean=CF.rename(columns={'location.lat': 'lat','location.lng':'long'})
CF_clean.head()

for lat, lng, label in zip(CF_clean.lat, CF_clean.long, CF_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)


-33.917 18.4244


KeyError: 'venues'

In [70]:
#4

address = 'City Centre, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

CC = json_normalize(venues)
CC.head()

CC_clean=CC.rename(columns={'location.lat': 'lat','location.lng':'long'})
CC_clean.head()

for lat, lng, label in zip(CC_clean.lat, CC_clean.long, CC_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)
CC_clean.head(20)

-33.9224221 18.4263523


KeyError: 'venues'

In [91]:
#5

address = 'Helderberg, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

Helder = json_normalize(venues)
Helder.head()

Helder_clean=Helder.rename(columns={'location.lat': 'lat','location.lng':'long'})
Helder_clean.head()

for lat, lng, label in zip(Helder_clean.lat, Helder_clean.long, Helder_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)




-34.0494444 18.8344444


<ipython-input-91-aa7102f7aaae>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Helder = json_normalize(venues)


In [92]:
#6

address = 'Northern Suburbs, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = -33.9775
longitude = 18.483
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

North_Suburbs = json_normalize(venues)
North_Suburbs.head()

North_Suburbs_clean=North_Suburbs.rename(columns={'location.lat': 'lat','location.lng':'long'})
North_Suburbs_clean.head()

for lat, lng, label in zip(North_Suburbs_clean.lat, North_Suburbs_clean.long, North_Suburbs_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)


-33.9775 18.483


<ipython-input-92-256d9b5ebae9>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  North_Suburbs = json_normalize(venues)


In [93]:
#7

address = 'Southern Suburbs, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = -34.0301
longitude = 18.451
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

South_Sub = json_normalize(venues)
South_Sub.head()

South_Sub_clean=South_Sub.rename(columns={'location.lat': 'lat','location.lng':'long'})
South_Sub_clean.head()

for lat, lng, label in zip(South_Sub_clean.lat, South_Sub_clean.long, South_Sub_clean.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)


-34.0301 18.451


<ipython-input-93-8b1fe61eb285>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  South_Sub = json_normalize(venues)


In [94]:
#8

address = 'South Peninsula, Cape Town'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = -33.9191
longitude = 18.422
print(latitude, longitude)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

venues = results['response']['venues']

South_Pen = json_normalize(venues)
South_Pen.head()

South_Pen_clean=South_Pen.rename(columns={'location.lat': 'lat','location.lng':'long'})
South_Pen_clean.head()

for lat, lng, label in zip(South_Pen_clean.lat, South_Pen_clean.long, South_Pen_clean.name,):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        label ='pharmacy',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(Pharmacy_map)


-33.9191 18.422


<ipython-input-94-c1d9bc15677f>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  South_Pen = json_normalize(venues)


In [95]:
# display map
Pharmacy_map

In [96]:
South_Pen.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bc3239ab492d13ac84ba860,Clicks Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,Burg St.,Church St.,-33.923634,18.419297,"[{'label': 'display', 'lat': -33.9236342445069...",1827,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Burg St. (Church St.), iKapa, 8000, iNingizim...",NaN
1,4c10feb181e976b001fe0feb,Hypermed Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"91 Main Road, Doric Crt, Green Point",York St,-33.908215,18.409575,"[{'label': 'display', 'lat': -33.9082145690918...",209,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[91 Main Road, Doric Crt, Green Point (York St...",NaN
2,5a5740f0364d9712b2a78a9b,Dis-Chem Pharmacy The Point Centre,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,76 Regent Road,NaN,-33.920890,18.383193,"[{'label': 'display', 'lat': -33.9208901, 'lng...",2640,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[76 Regent Road, iKapa, 8005, iNingizimu Afrika]",NaN
3,58a3bdb8c30f7936edefc851,Cape Quarter Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"27 Somerset Road, Cape Quarter, Green Point",NaN,-33.912914,18.416883,"[{'label': 'display', 'lat': -33.9129142761230...",844,8005,ZA,Green Point,Western Cape,iNingizimu Afrika,"[27 Somerset Road, Cape Quarter, Green Point, ...",NaN
4,4e16ec1ae4cd70c587bc6ae4,Waterfront Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,NaN,NaN,-33.903955,18.420108,"[{'label': 'display', 'lat': -33.9039547868217...",1258,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN


In [97]:
pharmacy.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bc3239ab492d13ac84ba860,Clicks Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,Burg St.,Church St.,-33.923634,18.419297,"[{'label': 'display', 'lat': -33.9236342445069...",1827,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Burg St. (Church St.), iKapa, 8000, iNingizim...",NaN
1,4c10feb181e976b001fe0feb,Hypermed Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"91 Main Road, Doric Crt, Green Point",York St,-33.908215,18.409575,"[{'label': 'display', 'lat': -33.9082145690918...",209,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[91 Main Road, Doric Crt, Green Point (York St...",NaN
2,5a5740f0364d9712b2a78a9b,Dis-Chem Pharmacy The Point Centre,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,76 Regent Road,NaN,-33.920890,18.383193,"[{'label': 'display', 'lat': -33.9208901, 'lng...",2640,8005,ZA,iKapa,Western Cape,iNingizimu Afrika,"[76 Regent Road, iKapa, 8005, iNingizimu Afrika]",NaN
3,58a3bdb8c30f7936edefc851,Cape Quarter Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,"27 Somerset Road, Cape Quarter, Green Point",NaN,-33.912914,18.416883,"[{'label': 'display', 'lat': -33.9129142761230...",844,8005,ZA,Green Point,Western Cape,iNingizimu Afrika,"[27 Somerset Road, Cape Quarter, Green Point, ...",NaN
4,4e16ec1ae4cd70c587bc6ae4,Waterfront Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1624821641,False,NaN,NaN,-33.903955,18.420108,"[{'label': 'display', 'lat': -33.9039547868217...",1258,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[iKapa, iNingizimu Afrika]",NaN


In [32]:
neighborhood_latitude = df.loc[0, 'Latitude1'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude1'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'district'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Seat: Cape Town are -33.91908999999998, 18.421980000000076.


In [33]:

LIMIT = 100 


radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=H1UCPKFE1SJC1OXGY5PQNG133KVSKYQCDRXMLFYE04AM00PO&client_secret=FOEUOF2Q5UAKRMOPG0IOGFHX0K3MTZ0C5IFQLAOJPR3PDG1X&v=20180604&ll=-33.91908999999998,18.421980000000076&radius=5000&limit=100'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d9de0e2da8845d36d5de46'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Cape Town',
  'headerFullLocation': 'Cape Town',
  'headerLocationGranularity': 'city',
  'totalResults': 185,
  'suggestedBounds': {'ne': {'lat': -33.87408995499994,
    'lng': 18.476107013015316},
   'sw': {'lat': -33.96409004500003, 'lng': 18.367852986984836}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc8e896762beee11eae3d38',
       'name': 'Cape Heritage Hotel Cape Town',
       'location': {'address': '90 Bree Street',
        'lat': -33.920829,
        'lng': 18.418326,
        'labeledLatLngs': [{'label': 'display',
          'lat': -33.920829,
          

In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-36-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Cape Heritage Hotel Cape Town,Hotel,-33.920829,18.418326
1,Hemelhuijs,Café,-33.918540,18.419911
2,Caroline's Fine Wine Cellar,Wine Shop,-33.920482,18.421648
3,The Gin Bar,Cocktail Bar,-33.922733,18.417150
4,Origin Coffee Roasting,Coffee Shop,-33.917169,18.417626


In [37]:
 print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
CT_districts = getNearbyVenues(names=df['district'],
                                   latitudes=df['Latitude1'],
                                   longitudes=df['Longitude1']
                                  )

Seat: Cape Town
Atlantic Seaboard
Blaauwberg
Cape Flats
City Centre
Helderberg
Northern Suburbs
Southern Suburbs
South Peninsula


In [41]:
print(CT_districts.shape)
CT_districts.head()

(666, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Seat: Cape Town,-33.91909,18.42198,Cape Heritage Hotel Cape Town,-33.920829,18.418326,Hotel
1,Seat: Cape Town,-33.91909,18.42198,Hemelhuijs,-33.918540,18.419911,Café
2,Seat: Cape Town,-33.91909,18.42198,Caroline's Fine Wine Cellar,-33.920482,18.421648,Wine Shop
3,Seat: Cape Town,-33.91909,18.42198,The Gin Bar,-33.922733,18.417150,Cocktail Bar
4,Seat: Cape Town,-33.91909,18.42198,Origin Coffee Roasting,-33.917169,18.417626,Coffee Shop


In [42]:
CT_districts.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Atlantic Seaboard,100,100,100,100,100,100
Blaauwberg,44,44,44,44,44,44
Cape Flats,47,47,47,47,47,47
City Centre,100,100,100,100,100,100
Helderberg,68,68,68,68,68,68
Northern Suburbs,12,12,12,12,12,12
Seat: Cape Town,100,100,100,100,100,100
South Peninsula,95,95,95,95,95,95
Southern Suburbs,100,100,100,100,100,100


In [44]:
print('There are {} uniques categories.'.format(len(CT_districts['Venue Category'].unique())))

There are 117 uniques categories.


In [45]:
# one hot encoding
CT_onehot = pd.get_dummies(CT_districts[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CT_onehot['Neighborhood'] = CT_districts['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CT_onehot.columns[-1]] + list(CT_onehot.columns[:-1])
CT_onehot = CT_onehot[fixed_columns]

CT_onehot.head()

,Neighborhood,African Restaurant,Airport,Airport Lounge,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tour Provider,Trail,Train,Train Station,Vineyard,Waterfront,Wine Bar,Wine Shop,Winery,Zoo
0,Seat: Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Seat: Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Seat: Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Seat: Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Seat: Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
CT_grouped = CT_onehot.groupby('Neighborhood').mean().reset_index()
CT_grouped

,Neighborhood,African Restaurant,Airport,Airport Lounge,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tour Provider,Trail,Train,Train Station,Vineyard,Waterfront,Wine Bar,Wine Shop,Winery,Zoo
0,Atlantic Seaboard,0.020000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.00,0.00,...,0.01,0.020000,0.000000,0.000000,0.000000,0.02,0.00,0.010000,0.000000,0.000000
1,Blaauwberg,0.022727,0.000000,0.000000,0.000000,0.00,0.000000,0.022727,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
2,Cape Flats,0.000000,0.021277,0.085106,0.000000,0.00,0.000000,0.000000,0.00,0.00,...,0.00,0.000000,0.000000,0.021277,0.000000,0.00,0.00,0.000000,0.000000,0.000000
3,City Centre,0.020000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.00,0.00,...,0.01,0.020000,0.000000,0.000000,0.000000,0.02,0.00,0.010000,0.000000,0.000000
4,Helderberg,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.014706,0.00,0.00,...,0.00,0.014706,0.000000,0.000000,0.029412,0.00,0.00,0.029412,0.014706,0.014706
5,Northern Suburbs,0.000000,0.000000,0.000000,0.000000,0.00,0.083333,0.000000,0.00,0.00,...,0.00,0.000000,0.083333,0.083333,0.000000,0.00,0.00,0.000000,0.083333,0.000000
6,Seat: Cape Town,0.020000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.00,0.00,...,0.01,0.020000,0.000000,0.000000,0.000000,0.02,0.00,0.010000,0.000000,0.000000
7,South Peninsula,0.000000,0.000000,0.000000,0.010526,0.00,0.000000,0.000000,0.00,0.00,...,0.00,0.031579,0.000000,0.010526,0.052632,0.00,0.00,0.000000,0.021053,0.000000
8,Southern Suburbs,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.02,0.01,...,0.00,0.010000,0.000000,0.000000,0.000000,0.00,0.02,0.000000,0.000000,0.000000


In [47]:
num_top_venues = 5

for hood in CT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CT_grouped[CT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Atlantic Seaboard----
         venue  freq
0        Hotel  0.12
1  Coffee Shop  0.08
2         Café  0.07
3         Park  0.03
4   Steakhouse  0.03


----Blaauwberg----
                venue  freq
0                Café  0.09
1  Seafood Restaurant  0.09
2          Restaurant  0.07
3      Breakfast Spot  0.07
4           Surf Spot  0.05


----Cape Flats----
                  venue  freq
0  Fast Food Restaurant  0.17
1         Grocery Store  0.09
2        Airport Lounge  0.09
3   Rental Car Location  0.09
4        Breakfast Spot  0.06


----City Centre----
                venue  freq
0               Hotel  0.12
1         Coffee Shop  0.08
2                Café  0.07
3  Seafood Restaurant  0.03
4                Park  0.03


----Helderberg----
                  venue  freq
0           Coffee Shop  0.10
1  Fast Food Restaurant  0.06
2            Restaurant  0.04
3    Seafood Restaurant  0.04
4     Convenience Store  0.04


----Northern Suburbs----
                venue  freq
0   Convenie

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CT_grouped['Neighborhood']

for ind in np.arange(CT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atlantic Seaboard,Hotel,Coffee Shop,Café,Park,Italian Restaurant,Theater,Steakhouse,African Restaurant,Gym,Market
1,Blaauwberg,Café,Seafood Restaurant,Breakfast Spot,Restaurant,Grocery Store,Beach,Burger Joint,Pizza Place,Shopping Mall,Gas Station
2,Cape Flats,Fast Food Restaurant,Rental Car Location,Airport Lounge,Grocery Store,Breakfast Spot,Coffee Shop,Seafood Restaurant,Gift Shop,Hotel,Burger Joint
3,City Centre,Hotel,Coffee Shop,Café,Steakhouse,Park,Theater,Seafood Restaurant,Brewery,Market,Italian Restaurant
4,Helderberg,Coffee Shop,Fast Food Restaurant,Convenience Store,Steakhouse,Restaurant,Shopping Mall,Seafood Restaurant,Wine Shop,Italian Restaurant,Vineyard
5,Northern Suburbs,Convenience Store,Shopping Mall,Train,Steakhouse,Fried Chicken Joint,Arts & Crafts Store,Seafood Restaurant,Train Station,Winery,Department Store
6,Seat: Cape Town,Hotel,Coffee Shop,Café,Seafood Restaurant,Park,Steakhouse,Theater,Tapas Restaurant,Hotel Bar,Ice Cream Shop
7,South Peninsula,Restaurant,Coffee Shop,Café,Steakhouse,Vineyard,Breakfast Spot,Fast Food Restaurant,Grocery Store,French Restaurant,Pizza Place
8,Southern Suburbs,Café,Coffee Shop,Pizza Place,Thai Restaurant,Burger Joint,Sushi Restaurant,Bakery,Italian Restaurant,Bar,Athletics & Sports


In [60]:
# set number of clusters
kclusters = 5

CT_grouped_clustering = CT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 0, 2, 1, 0, 2, 2], dtype=int32)